In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:

from google.colab import drive
drive.mount('/content/drive')
!ls
df = pd.read_csv('/content/drive/Shared drives/DS4A_2021/Team74/data/raw/public_150k_plus.csv')
df.head(3)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data


,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,9547507704,05/01/2020,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,Paid in Full,24,100,769358.78,769358.78,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,N,N,Existing or more than 2 years old,Sumter,SUMTER,SC,29150-9662,SC-05,62.0,325510.0,Unanswered,Unknown/NotStated,NaN,769358.78,NaN,NaN,NaN,NaN,NaN,Corporation,19248,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
1,9777677704,05/01/2020,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,NaN,Exemption 4,24,100,736927.79,736927.79,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,Y,Y,Existing or more than 2 years old,North Charleston,CHARLESTON,SC,29420-9000,SC-06,73.0,561730.0,White,Unknown/NotStated,NaN,736927.79,NaN,NaN,NaN,NaN,NaN,Sole Proprietorship,19248,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN
2,5791407702,05/01/2020,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,Paid in Full,24,100,691355.00,691355.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SEATTLE,KING,WA,98112-2922,WA-07,75.0,NaN,Unanswered,Unknown/NotStated,NaN,691355.00,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y


In [8]:
pd.set_option('display.max_columns', None)

In [9]:
df.dtypes

LoanNumber                       int64
DateApproved                    object
SBAOfficeCode                    int64
ProcessingMethod                object
BorrowerName                    object
BorrowerAddress                 object
BorrowerCity                    object
BorrowerState                   object
BorrowerZip                     object
LoanStatusDate                  object
LoanStatus                      object
Term                             int64
SBAGuarantyPercentage            int64
InitialApprovalAmount          float64
CurrentApprovalAmount          float64
UndisbursedAmount              float64
FranchiseName                   object
ServicingLenderLocationID        int64
ServicingLenderName             object
ServicingLenderAddress          object
ServicingLenderCity             object
ServicingLenderState            object
ServicingLenderZip              object
RuralUrbanIndicator             object
HubzoneIndicator                object
LMIIndicator             

In [10]:
df.shape

(964361, 51)

Convert Data type

In [21]:
df['DateApproved'] = pd.to_datetime(df['DateApproved'])
df['LoanStatusDate'] = pd.to_datetime(df['LoanStatusDate']) 
df['BorrowerZip'] = df['BorrowerZip'][:5]
#df['BorrowerZip'] = df['BorrowerZip']
# df = df.apply (pd.to_numeric, errors='coerce')
# df = df.dropna()
#df['ServicingLenderZip'] = df['ServicingLenderZip'].str.split('-').str[0].astype(int)
df.head(10)

AttributeError: ignored

In [15]:
)# df['BorrowerZip'] = df['BorrowerZip'].apply(pd.to_numeric, errors='coerce')
# df['BorrowerZip'] = df['BorrowerZip'].dropna()
zips = ((np.asarray(df['BorrowerZip'])))
result = df.pivot(index = 'BorrowerName', columns = 'BorrowerAddress', values = 'BorrowerZip')

#array to annotate heatmaps
labels = (np.asarray(["{0} \n {1:2f}".format(value)
                                             for value in zip(zips.flatten())]))

fig, ax = plt.subplots(figsize = (12,7))
title = "Heatmap by Zipcodes"
plt.title(title, fontsize = 18)
ttl = ax.titlettl.set_position([0.5,1.05])

print(result)
df.head(3)

ValueError: ignored

In [ ]:
duplicateDFRow = df[df.duplicated(['BorrowerName'])]
#duplicateDFRow = df[df.duplicated(['Name', 'Seasons'])]
print(duplicateDFRow)

        LoanNumber DateApproved  ...      Veteran NonProfit
97      7467618405   2021-02-12  ...   Unanswered         Y
105     5139728405   2021-02-07  ...   Unanswered       NaN
134     5038877006   2020-04-04  ...   Unanswered       NaN
139     8737768301   2021-01-30  ...   Unanswered         Y
140     8064027104   2020-04-15  ...   Unanswered       NaN
...            ...          ...  ...          ...       ...
964341  6210327002   2020-04-06  ...   Unanswered       NaN
964348  8257238307   2021-01-29  ...  Non-Veteran       NaN
964350  1222407410   2020-05-04  ...   Unanswered       NaN
964356  4395967002   2020-04-03  ...  Non-Veteran       NaN
964359  9054647103   2020-04-15  ...   Unanswered       NaN

[102728 rows x 51 columns]


In [ ]:
df.describe().apply(lambda s: s.apply('{0:.2f}'.format))

,LoanNumber,SBAOfficeCode,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,ServicingLenderLocationID,ServicingLenderZip,JobsReported,NAICSCode,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,OriginatingLenderLocationID
count,964361.00,964361.00,964361.00,964361.00,964361.00,964361.00,963978.00,964361.00,964361.00,964360.00,957718.00,336926.00,962535.00,46493.00,99663.00,23267.00,57580.00,31991.00,964361.00
mean,5424974790.99,571.60,36.18,100.00,532037.39,530459.29,12640.03,109308.50,47208.29,51.83,511683.15,15047.58,514638.50,47835.60,56136.31,6797.54,46757.94,14238.57,108927.27
std,2551194898.91,262.97,17.03,0.00,744459.81,738189.87,138146.10,134740.91,27754.84,67.49,181175.47,85255.12,714252.70,158609.70,111336.22,61613.80,102817.86,55271.43,136293.85
min,1000007108.00,101.00,19.00,100.00,0.00,150000.00,0.00,20.00,612.00,0.00,111110.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,20.00
25%,3271347106.00,373.00,24.00,100.00,200100.00,200320.00,0.00,29805.00,26003.00,18.00,336612.00,1.00,195500.00,0.00,13500.00,0.00,8000.00,0.00,29805.00
50%,5393928706.00,515.00,24.00,100.00,295000.00,294983.87,0.00,57328.00,43240.00,30.00,541211.00,1.00,286914.00,9500.00,28800.00,0.00,19350.50,400.00,56102.00
75%,7544017106.00,811.00,60.00,100.00,539917.00,539000.00,0.00,117723.00,71601.00,54.00,623110.00,4700.00,524100.00,40000.00,56198.50,0.00,43976.86,9639.07,116677.00
max,9999007109.00,1094.00,60.00,100.00,10000000.00,10000000.00,10000000.00,532950.00,99901.00,500.00,999990.00,10000000.00,10000000.00,10000000.00,5518277.93,2951590.00,3880000.00,2497617.00,532950.00


In [ ]:
df.head(3)

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,9547507704,2020-05-01,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150,2020-12-18,Paid in Full,24,100,769358.78,769358.78,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,N,N,Existing or more than 2 years old,Sumter,SUMTER,SC,29150-9662,SC-05,62.0,325510.0,Unanswered,Unknown/NotStated,NaN,769358.78,NaN,NaN,NaN,NaN,NaN,Corporation,19248,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
1,9777677704,2020-05-01,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420,NaT,Exemption 4,24,100,736927.79,736927.79,0.0,NaN,19248,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,Y,Y,Existing or more than 2 years old,North Charleston,CHARLESTON,SC,29420-9000,SC-06,73.0,561730.0,White,Unknown/NotStated,NaN,736927.79,NaN,NaN,NaN,NaN,NaN,Sole Proprietorship,19248,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN
2,5791407702,2020-05-01,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112,2021-03-17,Paid in Full,24,100,691355.00,691355.00,0.0,NaN,9551,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,SEATTLE,KING,WA,98112-2922,WA-07,75.0,NaN,Unanswered,Unknown/NotStated,NaN,691355.00,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,9551,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y


In [ ]:
df.corr()

,LoanNumber,SBAOfficeCode,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,ServicingLenderLocationID,ServicingLenderZip,JobsReported,NAICSCode,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,OriginatingLenderLocationID
LoanNumber,1.000000,-0.006890,-0.071816,NaN,0.008986,0.009299,-0.005356,-0.004462,-0.015997,-0.010427,-0.015748,0.025304,0.009077,-0.000280,0.041503,-0.008911,0.050705,0.018362,-0.003846
SBAOfficeCode,-0.006890,1.000000,0.008844,NaN,-0.006672,-0.006391,-0.007161,0.008453,0.668467,-0.000894,-0.032876,0.004391,-0.006433,0.075842,0.000166,0.079343,-0.009840,0.050537,0.015209
Term,-0.071816,0.008844,1.000000,NaN,-0.078508,-0.077501,0.120818,0.065306,0.011760,-0.086268,0.056398,-0.236767,-0.065179,-0.006953,-0.006927,0.062146,-0.023559,-0.013403,0.055107
SBAGuarantyPercentage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
InitialApprovalAmount,0.008986,-0.006672,-0.078508,NaN,1.000000,0.990098,0.120511,-0.013402,0.010256,0.751876,-0.051410,0.355976,0.984519,0.415490,0.641332,0.010751,0.676616,0.331566,-0.013685
CurrentApprovalAmount,0.009299,-0.006391,-0.077501,NaN,0.990098,1.000000,0.123360,-0.015075,0.010482,0.753893,-0.051469,0.356800,0.994429,0.416263,0.643789,0.013809,0.676842,0.330792,-0.015428
UndisbursedAmount,-0.005356,-0.007161,0.120818,NaN,0.120511,0.123360,1.000000,0.019752,-0.007555,0.086437,0.009052,-0.022442,0.129515,0.007873,0.018838,0.003871,0.005656,-0.000420,0.019947
ServicingLenderLocationID,-0.004462,0.008453,0.065306,NaN,-0.013402,-0.015075,0.019752,1.000000,-0.024860,-0.020299,0.027037,-0.024173,-0.012268,0.005789,0.023698,0.157909,0.013343,0.073922,0.974142
ServicingLenderZip,-0.015997,0.668467,0.011760,NaN,0.010256,0.010482,-0.007555,-0.024860,1.000000,-0.002629,-0.035393,0.017969,0.008116,0.137496,0.030902,0.065870,0.025737,0.084429,-0.023552
JobsReported,-0.010427,-0.000894,-0.086268,NaN,0.751876,0.753893,0.086437,-0.020299,-0.002629,1.000000,0.069949,0.260998,0.748823,0.331709,0.430805,0.004336,0.414391,0.259151,-0.020344


In [23]:
!git status


fatal: not a git repository (or any of the parent directories): .git
